In [ ]:
import json
import time
import requests
import random
from tqdm import tqdm

In [ ]:
# type = {
#     "clbg": "datatable7270063",
#     "hgyj": "datatable1062280",
#     "qscb": "datatable4594326",
#     "ggyb": "datatable6061074",
#     "xgyb": "datatable9433834",
#     "hyyb": "datatable3735394"
# }

# start_url1 = "http://reportapi.eastmoney.com/report/jg?cb={cb}&pageSize=50&beginTime=2018-04-03&" \
#                     "endTime={time}&pageNo={page}&fields=&qType={type}&orgCode=&author=&_={timestamp}"
        
# start_url2 = "http://reportapi.eastmoney.com/report/list?cb={cb}&industryCode=*&pageSize=50&" \
#                 "industry=*&rating=&ratingChange=&beginTime=2018-04-03&endTime={time}&pageNo={page}&" \
#                 "fields=&qType={type}&orgCode=&code=*&rcode=&_={timestamp}"

# start_url3 = "http://reportapi.eastmoney.com/report/newStockList?cb={cb}&pageSize=50&" \
#                 "beginTime=2018-04-03&endTime={time}&pageNo={page}&fields=&qType={type}&_={timestamp}"

# start_url4 = "http://reportapi.eastmoney.com/report/list?cb={cb}&industryCode=*&pageSize=50&" \
#                 "industry=*&rating=*&ratingChange=*&beginTime=2018-04-03&endTime={time}&pageNo={page}&" \
#                 "fields=&qType={type}&orgCode=&rcode=&_={timestamp}"


In [ ]:
def get_url(time_0='2022-03-22', time_1='2022-03-23', pages=100):
    url_list = []
    for page in range(1, pages+1):
        url = f"https://reportapi.eastmoney.com/report/jg?cb=datatable6176985&pageSize=100&beginTime={time_0}&endTime={time_1}&pageNo={page}"
        url_list.append(url)
    return url_list

def get_report(url_list):
    query_list = []
    for i in tqdm(range(len(url_list))):
        url = url_list[i]
        res = requests.get(url)
        res_text = res.text
        res_text = res_text[17:-1]
        res_js = json.loads(res_text)
        if len(res_js['data'])==0:
            break
        query_list.extend(res_js['data'])
        # time.sleep(random.uniform(1,2))
    return query_list

In [ ]:
url_list = get_url(time_0='2020-04-15', time_1='2022-04-15', pages=10000)

In [ ]:
query_list = get_report(url_list)

In [ ]:
import pickle

with open('./files/eastmoney_querys.pkl', 'wb') as f:
    pickle.dump(query_list, f)

In [ ]:
print(len(query_list))
print(query_list[-1])

In [ ]:
from bs4 import BeautifulSoup

def get_content(text_link):

    html = requests.get(text_link).content
    soup = BeautifulSoup(html, "lxml")

    # different classname
    # class_name = {
    #     '个股研报': 'newsContent', https://data.eastmoney.com/report/stock.jshtml, https://data.eastmoney.com/report/info/AP202204151559481399.html
    #     '新股研报': 'newsContent',
    #     '行业研报': 'ctx-content', https://data.eastmoney.com/report/industry.jshtml, 
    #     '宏观研究': 'ctx-content', https://data.eastmoney.com/report/macresearch.jshtml, 
    #     '策略报告': 'ctx-content', https://data.eastmoney.com/report/strategyreport.jshtml, 
    #     '券商晨报': 'ctx-content', https://data.eastmoney.com/report/brokerreport.jshtml, 
    # }

    res = {
        'ctx-content': soup.find('div', class_='ctx-content'),
        'newsContent': soup.find('div', class_='newsContent'),
    }
    
    for key in res.keys():
        if res[key] != None:
            content = [i.text for i in res[key].find_all('p')]
            content = ''.join(content)
            content = content.replace('\u3000\u3000', '')
            content = content.replace('\u3000\u3000', '')
            content = content.replace('\r', '')
            content = content.replace(' ', '')
            return key, content

    return None, None

In [47]:
query_list[0]['encodeUrl']

'mmD7f5NqJBTBwSg71cJspfA/SR3oQR9xI/e5QEHzkG4='

In [ ]:
# 单线程
for query in tqdm(query_list):
    encodeUrl = query['encodeUrl']
    content_url = f'https://data.eastmoney.com/report/zw_macresearch.jshtml?encodeUrl={encodeUrl}'
    key, content = get_content(content_url)
    query['content_url'] = content_url
    query['key'] = key
    query['content'] = content
    all.append(query)

In [2]:
import numpy as np

query_list = np.load('./files/eastmoney_querys.pkl', allow_pickle=True)
researchs = np.load('./files/eastmoney_researchs.pkl', allow_pickle=True)

In [38]:
print(len(query_list))

152291


In [3]:
len(researchs)

152291

In [39]:
num = 0
for item in researchs:
    if item['content']!=None:
        num = num + 1

In [40]:
num

152282

In [46]:
researchs[15]

{'id': 173000005436826449,
 'title': '锂电产业行业专题报告：隔膜产品日益紧俏，行业扩产节奏加快',
 'author': ['11000390722.刘杰'],
 'orgName': '英大证券有限责任公司',
 'orgCode': '80000085',
 'orgSName': '英大证券',
 'publishDate': '2022-04-15 00:00:00.000',
 'encodeUrl': 'mmD7f5NqJBTBwSg71cJspalesP36rHaAnRkNIi5jXME=',
 'researcher': '刘杰',
 'market': '',
 'industryCode': '1033',
 'industryName': '电池',
 'authorID': ['11000390722'],
 'orgType': 'white',
 'content_url': 'https://data.eastmoney.com/report/zw_macresearch.jshtml?encodeUrl=mmD7f5NqJBTBwSg71cJspalesP36rHaAnRkNIi5jXME=',
 'key': 'ctx-content',
 'content': '行业视点隔膜是锂电池关键原材料之一。隔膜是锂离子电池产业链中最具技术壁垒的关键内层组件。隔膜在电池中成本占比约为10%，隔膜的成本包括了原料成本、能源成本、人工成本及折旧摊销等。其中原料成本占比最大，约为41%；能源成本主要包括水电费用，占比18%；隔膜设备成本高，导致折旧摊销占比高，达到23%。隔膜是四大材料中格局最优的赛道。高壁垒下行业集中度高，龙头盈利能力强。由于行业壁垒较高，隔膜行业集中度高于其他锂电材料，2020年CR3达62%，CR6达80%，高于其余锂电材料。盈利能力上，隔膜毛利率也高于正极、负极、电解液，处于较高水平。锂电隔膜行业具备高度的行业壁垒。锂电池隔膜属于技术密集型行业，需要长期技术积累。隔膜项目需投入大量资金，一亿平隔膜项目投资在3.6亿元左右，设备投资占比在50%以上。目前隔膜设备均由隔膜厂商根据生产情况与设备厂商共同设计定制，头部隔膜企业已与主要隔膜设备厂商绑定。目前隔膜生产设备供应存在瓶颈，是目前隔膜扩